# <center>Gathering Supply Chain Data<center>

- Date: March 2019
- Author: Minsu Yeom, CFA, FRM

In [3]:
import os
import numpy as np
import pandas as pd
import bamboolib as bam
import xlwings as xw
from datetime import date
from mlfinance.utils.misc import get_letter, get_last_bday

In [1]:
# Reload all modules (except those excluded by %aimport) every time before executing the Python code typed.
%load_ext autoreload
%autoreload 2

# 1. Loading Equity Universe

In [3]:
universe = pd.read_table('./mlfinance/dataset/eqy_universe_gt50mm.txt',
                         header=1,
                         names=[
                             'Identifier', 'Name', 'Revenue', 'Company Type',
                             'Business Description', 'FactSet Industry',
                             'Crunchbase Category(BETA)',
                             'Crunchbase Rank(BETA)', 'Ultimate Parent Name',
                             'Fiscal Year End', 'Country', 'Website'
                         ], 
                         parse_dates = ['Fiscal Year End']
                        )

#### Change dtypes properly

In [4]:
universe.loc[:, 'Revenue'] = pd.to_numeric(universe.loc[:,'Revenue'], errors='coerce')
universe.loc[:, 'Fiscal Year End'] = pd.to_datetime(universe.loc[:, 'Fiscal Year End'], errors='coerce')

In [5]:
universe.dtypes

Identifier                           object
Name                                 object
Revenue                             float64
Company Type                         object
Business Description                 object
FactSet Industry                     object
Crunchbase Category(BETA)            object
Crunchbase Rank(BETA)                object
Ultimate Parent Name                 object
Fiscal Year End              datetime64[ns]
Country                              object
Website                              object
dtype: object

#### Setting the minimum market cap criteria in USD mm
- Reset the index.

In [6]:
min_market_cap = 200
universe = universe.loc[universe.loc[:, 'Revenue'] >= min_market_cap, :]
universe = universe.reset_index(drop = True)

### Missing value analysis

- Looks all good.

In [7]:
universe.isna().sum()

Identifier                     0
Name                           0
Revenue                        0
Company Type                   0
Business Description           0
FactSet Industry               0
Crunchbase Category(BETA)      0
Crunchbase Rank(BETA)          0
Ultimate Parent Name           0
Fiscal Year End                1
Country                        0
Website                      127
dtype: int64

In [8]:
universe.loc[pd.isna(universe['Fiscal Year End']), :]

,Identifier,Name,Revenue,Company Type,Business Description,FactSet Industry,Crunchbase Category(BETA),Crunchbase Rank(BETA),Ultimate Parent Name,Fiscal Year End,Country,Website
9383,ROCK-PH,Rockwell Land Corp.,297.71173,Public Company,Rockwell Land Corp. engages in the real estate...,Homebuilding,-,-,First Philippine Holdings Corporation,NaT,Philippines,http://www.e-rockwell.com


#### Launch Excel with making it sure FastSet add-ins enabled
- `fdswFixExcel.exe` enables the add-ins.
- `retcode`: 0 if runs fine. 1 if any error.

In [9]:
import subprocess
retcode = subprocess.run(['C:/Program Files (x86)/FactSet/fdswFixExcel.exe'])

In [20]:
subprocess.CompletedProcess.check_returncode

AttributeError: type object 'CompletedProcess' has no attribute 'returncode'

#### Load company tickers

In [11]:
# tickers = ['AAPL-US', 'GOOGL-US', 'AMZN-US']
tickers = universe.loc[0:999, 'Identifier'].to_list()

In [12]:
len(tickers)

1000

#### Set an Excel header

In [13]:
header_meta = ['date', 'ticker',  'relationship']
header_fcst = ['rel_comp_tic', 'rel_comp_nm', 'no_overlap', 'pct_overlap', 'revenue_dependence', 'source', 'prtr_rel']
header = header_meta + header_fcst

#### Set Excel column names and Excel ranges for a relationship data set

Column names

In [15]:
# Excel column names for meta data
meta_col = {}
for i, col_nm in enumerate(header_meta):
    meta_col[col_nm] = get_letter(i)

# Excel column names for FactSet data to be downloaded
fcst_start_col = get_letter(len(header_meta))
fcst_end_col = get_letter(len(header_fcst)-1, letter = fcst_start_col)

The whole data range

In [16]:
per_company_rows = 500
data_height = len(tickers) * per_company_rows
data_range = 'A1:' + get_letter(len(header)-1) + str(data_height)

#### FactSet settings

In [17]:
#wb=xw.books

# SUPL: Suppliers. 
# CUST: Customers
# PRTR: Partners
# COMP: Competitors
# It's FactSet's codes.
relationships = ['SUPL', 'CUST', 'PRTR', 'COMP']

# =FDS("GOOGL-US", "FF_COMPANY_RELATIONSHIP(SUPL,PUB,Ticker,ALL,ALL)")
fds_fn = '", "FF_COMPANY_RELATIONSHIP('
fds_param = ',PUB,Ticker,ALL,ALL)")'

#### Load company relationship data from FactSet and write it in an Excel workbook by
- Writing `=FDS()` directly an Excel sheet named as in `relationships`
- Allocating `rows` rows for each company

In [18]:
ws = {}
fcst_ranges = {}

# Iterate over ['SUPL', 'CUST', 'PRTR', 'COMP']
for i, rel in enumerate(relationships):
    ws[rel] = xw.sheets.add(rel)    # Create a sheet per relationship

    # Load company relationship data and write it in the workbook.
    for j, ticker in enumerate(tickers):
        fcst_ranges[ticker] = '{:s}{:d}:{:s}{:d}'.format(fcst_start_col, j*per_company_rows+1, fcst_end_col, (j+1)*per_company_rows)
        ws[rel].range(fcst_ranges[ticker]).formula_array = '=FDS("' + ticker + fds_fn + rel + fds_param
        ticker_col = get_letter(header_meta.index('ticker'))
        ticker_range = '{:s}{:d}:{:s}{:d}'.format(ticker_col, j*per_company_rows+1, ticker_col, (j+1)*per_company_rows)
        ws[rel].range(ticker_range).value = ticker


    for meta_type, col_nm in meta_col.items():
        meta_range = '{:s}{:d}:{:s}{:d}'.format(meta_col[meta_type], 1, meta_col[meta_type], data_height)
        meta_value = get_last_bday() if meta_type == 'date' else rel if meta_type == 'relationship' else '-'

        # We don't write `ticker` here because it's already filled.
        if meta_type != 'ticker':
            ws[rel].range(meta_range).value = meta_value

#### Extract data from each sheet in `ws` and merge it into `df`
- We also do: resetting the index and setting the column names.

In [26]:
ws_list = []
for sheet in ws.keys():
    ws_list.append(pd.DataFrame(ws[sheet].range(data_range).value))

df = pd.concat(ws_list)
df = df.reset_index(drop = True)
df.columns = header

In [32]:
df

,date,ticker,relationship,rel_comp_tic,rel_comp_nm,no_overlap,pct_overlap,revenue_dependence,source,prtr_rel
0,2020-03-20,DRI-DE,SUPL,TEF-ES,Telefonica SA,5 of 27,19,None,Direct,None
1,2020-03-20,DRI-DE,SUPL,VOD-GB,Vodafone Group Plc,4 of 31,13,None,Direct,None
2,2020-03-20,DRI-DE,SUPL,SAP-DE,SAP SE,0 of 93,0,None,Direct,None
3,2020-03-20,DRI-DE,SUPL,SGE-GB,Sage Group plc,0 of 22,0,None,Direct,None
4,2020-03-20,DRI-DE,SUPL,O2D-DE,Telefonica Deutschland Holding AG,4 of 6,67,None,Reverse,None
...,...,...,...,...,...,...,...,...,...,...
3999995,2020-03-20,8530-JP,COMP,None,None,None,None,None,None,None
3999996,2020-03-20,8530-JP,COMP,None,None,None,None,None,None,None
3999997,2020-03-20,8530-JP,COMP,None,None,None,None,None,None,None
3999998,2020-03-20,8530-JP,COMP,None,None,None,None,None,None,None


In [41]:
df.to_pickle('./mlfinance/dataset/company_relationships_1-1000.pkl')

In [4]:
df2 = pd.read_pickle('./mlfinance/dataset/company_relationships_1-1000.pkl')

In [5]:
df2

,date,ticker,relationship,rel_comp_tic,rel_comp_nm,no_overlap,pct_overlap,revenue_dependence,source,prtr_rel
0,2020-03-20,DRI-DE,SUPL,TEF-ES,Telefonica SA,5 of 27,19,None,Direct,None
1,2020-03-20,DRI-DE,SUPL,VOD-GB,Vodafone Group Plc,4 of 31,13,None,Direct,None
2,2020-03-20,DRI-DE,SUPL,SAP-DE,SAP SE,0 of 93,0,None,Direct,None
3,2020-03-20,DRI-DE,SUPL,SGE-GB,Sage Group plc,0 of 22,0,None,Direct,None
4,2020-03-20,DRI-DE,SUPL,O2D-DE,Telefonica Deutschland Holding AG,4 of 6,67,None,Reverse,None
...,...,...,...,...,...,...,...,...,...,...
3999995,2020-03-20,8530-JP,COMP,None,None,None,None,None,None,None
3999996,2020-03-20,8530-JP,COMP,None,None,None,None,None,None,None
3999997,2020-03-20,8530-JP,COMP,None,None,None,None,None,None,None
3999998,2020-03-20,8530-JP,COMP,None,None,None,None,None,None,None


In [6]:
df2.loc[df2.rel_comp_tic=='Reverse', :]

,date,ticker,relationship,rel_comp_tic,rel_comp_nm,no_overlap,pct_overlap,revenue_dependence,source,prtr_rel
7005,2020-03-20,AFE-ZA,SUPL,Reverse,None,None,None,None,None,None
9005,2020-03-20,7745-JP,SUPL,Reverse,None,None,None,None,None,None
17005,2020-03-20,AAK-SE,SUPL,Reverse,None,None,None,None,None,None
20005,2020-03-20,ARL-DE,SUPL,Reverse,None,None,None,None,None,None
38005,2020-03-20,ADIB-AE,SUPL,Reverse,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...
3954005,2020-03-20,3320-HK,COMP,Reverse,None,None,None,None,None,None
3955005,2020-03-20,836-HK,COMP,Reverse,None,None,None,None,None,None
3958005,2020-03-20,3838-HK,COMP,Reverse,None,None,None,None,None,None
3981005,2020-03-20,116-HK,COMP,Reverse,None,None,None,None,None,None


In [7]:
df3 = df2.loc[:, ['ticker', 'relationship', 'rel_comp_tic']]

In [8]:
df3
# bamboolib live code export
df3 = df3.dropna(subset=['rel_comp_tic'])
df3 = df3.groupby(['ticker', 'relationship']).agg(rel_comp_tic_nunique=('rel_comp_tic', 'nunique')).reset_index()
df3

,ticker,relationship,rel_comp_tic
0,DRI-DE,SUPL,TEF-ES
1,DRI-DE,SUPL,VOD-GB
2,DRI-DE,SUPL,SAP-DE
3,DRI-DE,SUPL,SGE-GB
4,DRI-DE,SUPL,O2D-DE
...,...,...,...
3999995,8530-JP,COMP,None
3999996,8530-JP,COMP,None
3999997,8530-JP,COMP,None
3999998,8530-JP,COMP,None


TypeError: unsupported operand type(s) for -: 'NoneType' and 'int'

In [42]:
df.to_feather('./mlfinance/dataset/company_relationships_1-1000.feather')

ImportError: Missing optional dependency 'pyarrow'.  Use pip or conda to install pyarrow.

## ----------------- The end of work -------------------

#### Set ranges in each sheet for each company in `tickers`
- e.g. `cell_ranges['AAPL-US'] <- 'D1:J1000'

In [9]:
height = 1000
fcst_data_ranges = {}
for i, name in enumerate(rel_names):
    print(i+1, name)

for j, ticker in enumerate(tickers):
    fcst_data_ranges[ticker] = 'A{:d}:G{:d}'.format(j*height+1, (j+1)*height)
    print(fcst_data_ranges[ticker])
#     print('A{:d}:G{:d}'.format(j*height+1, (j+1)*height))

NameError: name 'rel_names' is not defined

#### `xw.App` launches Excel.
- With `visible` False, it runs in the background

In [ ]:
app = xw.App(visible=True)

In [ ]:
xw.books

In [ ]:
bk

In [ ]:
book = app.books.open(os.getcwd() + '/test.xlsx')
sheet = book.sheets['Suppliers']
sheet.range('A1').value = 12

In [ ]:
sheets = {}
sheets['customers'] = book.sheets['Customers']
sheets['suppliers'] = book.sheets['Suppliers']
sheets['Partners'] = book.sheets['Partners']

In [ ]:
sheets['customers']

In [ ]:
import win32com.client as win32
from win32com.client import makepy

In [ ]:
path = 'C:/Program Files (x86)/FactSet/fdsxl32_x64.xll'
xl = win32.gencache.EnsureDispatch('Excel.Application')
xl.Visible = True
xl.DisplayAlerts=False
xl.RegisterXLL(path)
# xl.COMAddIns("PowerlinkCOMAddIn.COMAddIn").Connect = True
#xl.COMAddIns('C:/Program Files (x86)/FactSe|t/COMShim_x64.dll').Installed = True
#xl.COMAddIns(4)
for i in range(1, xl.COMAddIns.Count+1):
    print("Index: ", i, xl.COMAddIns(i).Description, xl.COMAddIns(i).Connect)
    if xl.COMAddIns(i).Description == "FactSet Office COM Add-In":
        print("xl.COMAddIns({:d}).Description: {:s}".format(i, xl.COMAddIns(i).Description))

wb = xl.Workbooks.Open(os.getcwd() + '/test.xlsx')
ws = wb.Worksheets('Sheet1')

In [ ]:
xl.COMAddIns.Count

In [ ]:
import subprocess
import xlwings as xw
retcode = subprocess.call("C:/Program Files (x86)/FactSet/fdswFixExcel.exe")
#print(retcode)
#xl = win32.gencache.EnsureDispatch('Excel.Application')
#xl.Visible = True
#xl.DisplayAlerts = False
print(xw.books.active.name)
#wb = xl.Workbooks.Open(os.getcwd() + '/test.xlsx')
#ws = wb.Worksheets('Sheet1')

In [ ]:
import xlwings

In [ ]:
xl2 = win32.gencache.EnsureDispatch("Excel.Application")
xl2.Visible = True
wb2 = xl.Workbooks.Add(1)
xl2.AddIns("FactSet Office COM Add-In").Installed = False
xl2.AddIns("FactSet Office COM Add-In").Installed = True

In [ ]:
xl.run('FDS', 'a', 'b', 'c', 'd')